In [2]:
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs

/Users/3arthzneiz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [22]:
input_file = codecs.open("socialmedia_relevant_cols.csv", "r",encoding='utf-8', errors='replace')
output_file = open("socialmedia_relevant_cols_clean.csv", "w")


def sanitize_characters(raw, clean):    
    for line in input_file:
        # remove sharp from line
        out = line.replace('#', '')
        output_file.write(out)
sanitize_characters(input_file, output_file)

In [25]:
questions = pd.read_csv("socialmedia_relevant_cols_clean.csv")
questions.columns=['text', 'choose_one', 'class_label']
questions.head(15)

,text,choose_one,class_label
0,Just happened a terrible car crash,Relevant,1
1,Our Deeds are the Reason of this earthquake Ma...,Relevant,1
2,"Heard about earthquake is different cities, st...",Relevant,1
3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,Forest fire near La Ronge Sask. Canada,Relevant,1
5,All residents asked to 'shelter in place' are ...,Relevant,1
6,"13,000 people receive wildfires evacuation ord...",Relevant,1
7,Just got sent this photo from Ruby Alaska as s...,Relevant,1
8,RockyFire Update => California Hwy. 20 closed ...,Relevant,1
9,Apocalypse lighting. Spokane wildfires,Relevant,1


In [ ]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

questions = standardize_text(questions, "text")

questions.to_csv("clean_data.csv")